Big City Crime Analysis

by: Cameron Sudderth, Ayana Rodgers, Moni Buddha

https://github.com/aya-rodgers97/Crime-Project-Tools-1


2024 is a Presidential Election year and crime is always a major topic with which Americans vote, both on a Federal and Local level​

Every major US city publishes data related to its crimes, so we wanted to look at the major metro areas in the US to see what similarities exist

API Connections​:

NYC Crime Data​
City of LA Crime Data​
City of Chicago Crime Data​

Categorize the Crime Descriptions into commonly  themed categories​

Identify the most significant crimes in each city​

Identify the most affected gender in each city.​

To find patterns in each season, day of the week and hour of the day in each city.

For research, there were many articles on the analysis of crime throughout different cities. One article we used is Los Angeles Neighborhood Analysis by Chaitany Krishna Kasaraneni on Towards Data Science. In this article they used crime to try and select the best locations in Los Angeles to possibly live or open a business. In this article, data was collected through data scraping for the Latitude and Longitude of crime in the city. Also, a dataset of Los Angeles neighborhoods was used to determine safest areas. Through this article we got the idea to map our crimes and try to determine criminal hot spots in each of our cities. 

Additionally, there is a book Crime Analysis with Crime Mapping by Rachel Boba Santos. Mainly, looking at the summary this is an entire book based on Crime Analysis with foundation, process,
data, different ways that crime can be analyzed. Crime analysis is used to support major funtions within police agencies. Such as analyzing calls for service, quality of life issues, traffic craches. With such analyzes agencies can determine where there should be more police presence, those with greater incidents. Through our project we seek to perform such analysis to determine the higher rates of crime in areas of the cities, time of day, season, etc. 


In [1]:
import requests
import pandas as pd

NYC API Connection

In [2]:
import pandas as pd
from sodapy import Socrata

# Number of Records limit
nyc_limit = 500
la_limit = 500
chi_limit = 500

# Connect to NYC API
NYCAppToken = '3u5hcZ6WwKere5Mb5nm5S9mT2'
nyc_client = Socrata("data.cityofnewyork.us",
                 NYCAppToken,
                 username="Cameron.Suddreth@du.edu",
                 password="COMP4447groupproject")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
nyc_results = nyc_client.get("5uac-w243", limit=nyc_limit)

# Convert to pandas DataFrame
nyc_df = pd.DataFrame.from_records(nyc_results)


LA API Connection

In [3]:

# Example authenticated client (needed for non-public datasets):
LA_AppToken = 'mEU8HkgWCvfkWLHKGxfiUFecc'
la_client = Socrata("data.lacity.org",
                 LA_AppToken,
                 username="Cameron.Suddreth@du.edu",
                 password="COMP4447groupproject")


la_results = la_client.get("2nrs-mtv8", limit=la_limit)

# Convert to pandas DataFrame
la_df = pd.DataFrame.from_records(la_results)

Chicago API Connection

In [4]:
CHI_AppToken = '6rxQVr5BfXAbUUccKTodxYVdj'
chi_client = Socrata("data.cityofchicago.org",
                    CHI_AppToken,
                    username="Cameron.Suddreth@du.edu",
                    password="COMP4447groupproject")
chi_results = chi_client.get("9hwr-2zxp", limit=chi_limit)
chi_df = pd.DataFrame.from_records(chi_results)

Above, we have established three different API connections. We have established an API connection with each of the three largest cities in the United States: New York, NY; Los Angeles, CA; and Chicago, IL. We are also able to change the number of records that we are pulling in for analysis as each of the databases combined would result in close to one million records!

With each connection, we have created a separate dataframe which allowed us to easily pull in all the records from each city's database. As we now have each of the cities with their own dataframe, we will begin to merge the dataframes together to allow us to look at the data amongst the cities together.

The below cell compares the number of columns within each dataframe. This was the initial step for us to begin merging the dataset together.

In [5]:
nyc_columns = nyc_df.columns
la_columns = la_df.columns
chi_columns = chi_df.columns

print(f'Number of NYC Columns: {len(nyc_columns)}')
print(f'Number of LA Columns: {len(la_columns)}')
print(f'Number of CHI Columns: {len(chi_columns)}')

Number of NYC Columns: 40
Number of LA Columns: 26
Number of CHI Columns: 22


We reviewed the column names within the API documentation and determined what information we wanted from each city and then reduced the size of each dataframe to contain only those columns which we wanted to analyze. We then renamed each column so to allow for easier analysis and merging.

Before we create the new column headers, we must also resolve any NAN values else we will receive an error. We have imported the Numpy module to fill in the Weapon user for each NYC and Chicago; the time of the crime in Chicago; and the victim sex for Chicago as this data was unavailable in each respective city's database.

We have also created a new column in each city's original dataframe to help identify which city each record belongs to after we have merged the data.

In [6]:
import numpy as np

nyc_df['Weapon'] = np.nan
chi_df['Weapon'] = np.nan
chi_df['Time'] = np.nan
chi_df['Victim Sex'] = np.nan
nyc_df['City'] = 'NYC'
la_df['City'] = "LA"
chi_df['City'] = 'CHI'

la_df = la_df[['dr_no', 'date_rptd', 'date_occ', 'crm_cd', 'crm_cd_desc', 
'weapon_desc', 'vict_sex', 'lat', 'lon', 'City']]
nyc_df = nyc_df[['cmplnt_num', 'cmplnt_fr_dt', 'cmplnt_fr_tm','ky_cd', 'ofns_desc', 
'Weapon', 'vic_sex', 'latitude','longitude', 'City']]
chi_df = chi_df[['id', 'date','Time', 'iucr','description','Weapon', 
'Victim Sex','latitude','longitude', 'City']]
generic_columns = ['Case Number', 'Date', 'Time', 'Crime Code', 'Crime Description','Weapon', 
'Victim Sex', 'Latitude', 'Longitude', 'City']

# Rename the columns
la_df.columns = generic_columns
nyc_df.columns = generic_columns
chi_df.columns = generic_columns

By renaming the columns of the dataframe above, it simplified the merging process, so we did not have to specify what column in each dataframe to merge based on.

In [7]:
combined_df = pd.concat([la_df, nyc_df, chi_df],ignore_index=True)
print(len(combined_df))


1500


In [8]:
combined_df['Crime Description'].sample(25)

309                          THEFT OF IDENTITY
126     CRIMINAL THREATS - NO WEAPON DISPLAYED
396                          THEFT OF IDENTITY
695               ASSAULT 3 & RELATED OFFENSES
1246                              TO RESIDENCE
79                           THEFT OF IDENTITY
1051                                    SIMPLE
684             OFFENSES AGAINST PUBLIC ADMINI
434                   BATTERY - SIMPLE ASSAULT
764               ASSAULT 3 & RELATED OFFENSES
151                            BRANDISH WEAPON
560                              HARRASSMENT 2
1498                                   TO LAND
1289                            $500 AND UNDER
872                              PETIT LARCENY
1445                           ARMED - HANDGUN
142                          THEFT OF IDENTITY
945                              HARRASSMENT 2
1160                                 OVER $500
246                          THEFT OF IDENTITY
980                             FELONY ASSAULT
590          

In [9]:
combined_df['str_length'] = combined_df['Crime Description'].str.len()
number_of_crimes = combined_df['Crime Description'].value_counts()
list_of_crimes = combined_df['Crime Description'].unique().tolist()
list_of_crimes_lower = [crime.lower() for crime in list_of_crimes]

print(len(list_of_crimes))

152


In [10]:
# !pip install transformers
# !pip3 install torch torchvision

Simplify the Descriptions

In [11]:
# !pip install spacy

In [12]:
combined_df[['Crime Description', 'City']].sample(20)

,Crime Description,City
850,THEFT-FRAUD,NYC
754,HARRASSMENT 2,NYC
1285,TO PROPERTY,CHI
968,UNAUTHORIZED USE OF A VEHICLE,NYC
1294,TO VEHICLE,CHI
1452,SIMPLE,CHI
1060,RECKLESS FIREARM DISCHARGE,CHI
969,GRAND LARCENY,NYC
242,THREATENING PHONE CALLS/LETTERS,LA
1209,SIMPLE,CHI


In [13]:
import sys
import re
try:
    # Try to import the SpaCy model to check if it's installed
    import en_core_web_sm
except ImportError:
    # If the model is not installed, download it using the subprocess module
    print("Downloading the 'en_core_web_sm' model...")
    !python3.7 -m spacy download en_core_web_sm

import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Remove digits and punctuation using a regular expression
    text = re.sub(r'[\d]', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation except underscores and spaces
    return text.lower()

def lemmatize(text):
    # Apply spaCy nlp pipeline to pre-processed text
    doc = nlp(text)
    # Return lemmatized text, only including tokens that are alphabetic
    return " ".join([token.lemma_ for token in doc if token.is_alpha])

# Assuming 'combined_df' is your DataFrame and 'Lemmatized Text' is the column to process
# Step 1: Pre-process the text to remove digits and punctuation
combined_df['Preprocessed Text'] = combined_df['Crime Description'].apply(preprocess_text)

# Step 2: Apply lemmatization
combined_df['Lemmatized Text'] = combined_df['Preprocessed Text'].apply(lemmatize)

# Display the processed DataFrame
print(combined_df[['Lemmatized Text']].head())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


               Lemmatized Text
0                vehicle steal
1        burglary from vehicle
2                   bike steal
3  shopliftinggrand theft over
4            theft of identity


In [52]:
import sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import pandas as pd  # Assuming you have a DataFrame 'combined_df'

def run_lda(n_topics, max_df, min_df, combined_df, top_topicwords):

    custom_stop_words = ["ord", "yrs", "pub", "unlawful", "petty", "lewd", "grand", "chld", "forcible", "relate",
     "susp", "agnst", "felony", "miscellaneous", "mischief", "criminal", "crm", "bunco", "old", "law", "letter", "telephone", "overexcptgunsfowllivestkprod"]
    stop_words = list(ENGLISH_STOP_WORDS.union(custom_stop_words))
    cv = CountVectorizer(max_df=max_df, min_df=min_df, stop_words=stop_words, lowercase=True)
    cv.fit(combined_df['Lemmatized Text'])
    dtm = cv.transform(combined_df['Lemmatized Text'])
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(dtm)

    top_words_per_topic = []
    for index, topic in enumerate(lda.components_):
        # note may need to change to cv.get_feature_names() for older versions of sklearn
        top_words = [cv.get_feature_names_out()[i] for i in topic.argsort()[-top_topicwords:]]
        top_words_per_topic.append(top_words)
        # print(f'Top {top_topicwords} words for topic {index}: {top_words}')
    
    # Transform the DTM to get the topic results
    topic_results = lda.transform(dtm)

    # Return the perplexity and top words for each topic
    return lda.perplexity(dtm), top_words_per_topic, topic_results

# Initialize variables to keep track of the best model
lowest_perplexity = float('inf')
best_model_config = None
best_top_words = None
best_topic_results = None

# Parameters to iterate over
n_topics_options = [5, 7, 10]
max_df_options = [0.7, 0.75, 0.8, 0.85]
min_df_options = [2, 5]
best_num_top_words = 3

for n_topics in n_topics_options:
    for max_df in max_df_options:
        for min_df in min_df_options:
            perplexity, top_words, topic_results = run_lda(n_topics, max_df, min_df, combined_df, best_num_top_words)
            if perplexity < lowest_perplexity:
                lowest_perplexity = perplexity
                best_model_config = (n_topics, max_df, min_df)
                best_top_words = top_words
                best_topic_results = topic_results

# Use the best model to append the dominant topic's top word to each document in the DataFrame
dominant_topics = best_topic_results.argmax(axis=1)
combined_df['Dominant Topic Word'] = [best_top_words[topic][0] for topic in dominant_topics]  # Using the first top word as representative
print(best_top_words)
print(f'Best Model Configuration: {best_model_config} with Lowest Perplexity: {lowest_perplexity}')

[['domestic', 'battery', 'simple'], ['property', 'petit', 'larceny'], ['retail', 'identity', 'theft'], ['plain', 'build', 'theft'], ['automobile', 'assault', 'harrassment'], ['aggravate', 'vehicle', 'traffic'], ['partner', 'intimate', 'sensblty'], ['sex', 'crime', 'burglary'], ['steal', 'motor', 'vehicle'], ['rape', 'possession', 'handgun']]
Best Model Configuration: (10, 0.7, 5) with Lowest Perplexity: 37.469892633513716


Looking at the below sample of original crime descriptions compared to the dominant topic word that is being assigned, you can see that some of them do not necessarily make sense. An area where this can be improved for future use is by continuing to run the model and adding descriptive words that do not make sense to the stop words list. Another important observation is that the topics holistically appear to be getting closer to characterizing each crime description; however, the "dominant topic word" is the first word within the list of the top 3 words for each topic, which is not necessarily correctly describing the original crime description.

Looking at an article on Towards Data Science, it was seen that perplexity is a good measure of how accurate the model is working. The lower the perplexity, the better the ability to predict the next word. Perplexity of around 37 is the lowest value that we have gotten and spot checking the records appears to be getting closer to accurately selecting a new crime description.

Source: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [59]:
print(combined_df[['Crime Description', 'Dominant Topic Word']].sample(10))
print(combined_df['Dominant Topic Word'].unique())

                                      Crime Description Dominant Topic Word
1373                                  CREDIT CARD FRAUD              retail
403                                   THEFT OF IDENTITY              retail
414                                   THEFT OF IDENTITY              retail
1474                                     FORCIBLE ENTRY             partner
1001                                         AUTOMOBILE          automobile
350                                          PICKPOCKET            domestic
1024                                             SIMPLE            domestic
518                                       PETIT LARCENY            property
751                                       HARRASSMENT 2          automobile
316   THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...            property
['steal' 'retail' 'domestic' 'sex' 'rape' 'automobile' 'property' 'plain'
 'partner' 'aggravate']


Graphing the crimes and their cities

In [16]:
import folium
from folium import plugins

With this first graph you can see when we mass graph all of the crimes in their respective cities it's difficult to interperet which are the hot crime areas or not. It'll be better to interpret if we grouped the points in their areas. 

In [17]:
df = combined_df 
 
# Convert Latitude and Longitude to float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
 
# Create a map centered around the mean of latitude and longitude
map1 = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=10)
 
# Add markers for each crime location
for index, row in df.iterrows():
    # Check for NaN values
    if not pd.isnull(row['Latitude']) and not pd.isnull(row['Longitude']):
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['Dominant Topic Word']).add_to(map1)

# Save the map
filename = 'crime_map.html'  # Specify a full file path
#map1.save(filename)
map1

In the map below we have clustered the markers together. This helps to determine the hot spots in each city. When observing it appears that most of the crimes occur in south of Chicago, centralized west of Los Angeles, and north in Staten Island of New York. Additionally, once the markers are clicked there is a popup to determine what crime has occurred there.

In [18]:
la_lon = -118.2426
la_lat = 34.0549
#Create a second map with the same LA coordinates
la_map2 = folium.Map(location = [la_lat, la_lon], zoom_start = 9)

#Instantiate a mark cluster object for the incidents in the dataframe
incidents2 = plugins.MarkerCluster().add_to(la_map2)

l_df = df.rename(columns={'Dominant Topic Word': 'Crime'})

l_df = l_df.dropna(subset=['Longitude','Latitude','Crime'])
#print(l_df)

#print(type(lat))
#Loop through the dataframe and add each data point to the mark cluster
for lat, lng, crm, in zip(l_df.Latitude, l_df.Longitude, l_df.Crime):
    
    folium.Marker(
        location=[lat, lng],
        popup=crm,
        icon=None,
    ).add_to(incidents2)

#Display map
la_map2